In [1]:
import os

os.chdir('../..')

import utils

In [2]:
outputfolder = os.getcwd() + '/output/'
datafolder = os.getcwd() + '/datasets/PTB-XL/'
sampling_rate = 100
task = 'priority'
experiment_name = 'exp4'

data, raw_labels = utils.load_dataset(datafolder, sampling_rate=sampling_rate)
labels = utils.compute_label_aggregations(raw_labels, datafolder, task)
data, labels, Y, _ = utils.select_data(data, labels, task, 0, outputfolder+experiment_name+'/data/')

In [3]:
X_test = data[labels.strat_fold == 10]
y_test = Y[labels.strat_fold == 10]

X_val = data[labels.strat_fold == 9]
y_val = Y[labels.strat_fold == 9]

X_train = data[labels.strat_fold <= 8]
y_train = Y[labels.strat_fold <= 8]

n_classes = y_train.shape[1]

print(f"This experiment has {n_classes} classes")

X_train_lead1 = X_train[:,:,0]
X_test_lead1 = X_test[:,:,0]
X_val_lead1 = X_val[:,:,0]

This experiment has 43 classes


In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from tqdm import tqdm
import pickle
import numpy as np
from tqdm import tqdm
import os
import time
import numpy as np
import pandas as pd
import scipy.io as sio
from scipy.fftpack import fft
from sklearn.ensemble import RandomForestClassifier
import pywt
import scipy.stats
import multiprocessing
import datetime as dt
from collections import defaultdict, Counter
from tensorflow.keras.layers import Dropout, Dense, Input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import backend as K
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from pysiology.electrocardiography import analyzeECG
from utils import CustomMetric as CM

Thank you for using Pysiology. If you use it in your work, please cite:
Gabrieli G., Azhari A., Esposito G. (2020) PySiology: A Python Package for Physiological Feature Extraction. In: Esposito A., Faundez-Zanuy M., Morabito F., Pasero E. (eds) Neural Approaches to Dynamics of Signal Exchanges. Smart Innovation, Systems and Technologies, vol 151. Springer, Singapore. https://doi.org/10.1007/978-981-13-8950-4_35


In [5]:
def calculate_entropy(list_values):
    counter_values = Counter(list_values).most_common()
    probabilities = [elem[1]/len(list_values) for elem in counter_values]
    entropy = scipy.stats.entropy(probabilities)
    return entropy

def calculate_statistics(list_values):
    n5 = np.nanpercentile(list_values, 5)
    n25 = np.nanpercentile(list_values, 25)
    n75 = np.nanpercentile(list_values, 75)
    n95 = np.nanpercentile(list_values, 95)
    median = np.nanpercentile(list_values, 50)
    mean = np.nanmean(list_values)
    std = np.nanstd(list_values)
    var = np.nanvar(list_values)
    rms = np.nanmean(np.sqrt(list_values**2))
    return [n5, n25, n75, n95, median, mean, std, var, rms]

def calculate_crossings(list_values):
    zero_crossing_indices = np.nonzero(np.diff(np.array(list_values) > 0))[0]
    no_zero_crossing_indices = len(zero_crossing_indices)
    mean_crossing_indices = np.nonzero(np.diff(np.array(list_values) < np.nanmean(list_values)))[0]
    no_mean_crossings = len(mean_crossing_indices)
    return [no_zero_crossing_indices, no_mean_crossings]

def get_features(list_values):
    entropy = calculate_entropy(list_values)
    crossings = calculate_crossings(list_values)
    statistics = calculate_statistics(list_values)
    return [entropy] + crossings + statistics

def psy_features(channel):
    dic = analyzeECG(channel, samplerate=100, pnn50pnn20=False, freqAnalysis=False,
                                             freqAnalysisFiltered=False)
    ibi = [dic['ibi']/100]
    val_names = ['bpm', 'sdnn', 'sdsd', 'rmssd', 'pnn50', 'pnn20']
    vals = [dic[val] for val in val_names]
    pnn50pnn20 = [0 if vals[-1]==0 else vals[-2]/vals[-1]]
    return ibi + vals + pnn50pnn20
        

def get_single_ecg_features(signal, waveletname='db6'):
    features = []
    for channel in signal.T:
        list_coeff = pywt.wavedec(channel, wavelet=waveletname, level=5)
        channel_features = []
        for coeff in list_coeff:
            channel_features += get_features(coeff)
        
        channel_features += psy_features(channel)
        
        features.append(channel_features)
    return np.array(features).flatten()

def get_ecg_features(ecg_data, parallel=True):
    if parallel:
        pool = multiprocessing.Pool(4)
        return np.array(pool.map(get_single_ecg_features, ecg_data))
    else:
        list_features = []
        for signal in tqdm(ecg_data):
            features = get_single_ecg_features(signal)
            list_features.append(features)
        return np.array(list_features)

In [27]:
X_train_lead1 = X_train[:,:,0].reshape(X_train.shape[0], X_train.shape[1], 1)
X_val_lead1 = X_val[:,:,0].reshape(X_val.shape[0], X_train.shape[1], 1)

In [28]:
X_train1 = get_ecg_features(X_train_lead1, parallel=False)
X_val1 = get_ecg_features(X_val_lead1, parallel=False)

100%|██████████| 2135/2135 [00:14<00:00, 142.80it/s]


In [29]:
from sklearn.preprocessing import PolynomialFeatures

X_train1 = PolynomialFeatures().fit_transform(X_train1)
X_val1 = PolynomialFeatures().fit_transform(X_val1)

In [37]:
input_x = Input(shape=(X_train1.shape[1],))
x = Dense(4096, activation="relu")(input_x)
x = Dense(2048, activation="relu")(x)
x = Dense(1024, activation="relu")(x)
x = Dense(512, activation="relu")(x)
x = Dense(512, activation="relu")(x)
x = Dense(256, activation="relu")(x)
x = Dense(256, activation="relu")(x)
x = Dense(128, activation="relu")(x)
x = Dense(128, activation="relu")(x)
x = Dense(64, activation="relu")(x)
x = Dense(64, activation="relu")(x)
y = Dense(43, activation="sigmoid")(x)
model = Model(input_x, y)

In [38]:
model.compile(optimizer='adam', loss="binary_crossentropy", 
              metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy', threshold=0.5),
                           tf.keras.metrics.Recall(name='Recall'),
                           tf.keras.metrics.Precision(name='Precision'),
                           tf.keras.metrics.AUC(num_thresholds=200, curve="ROC", name="AUC",
                                                multi_label=True, label_weights=None)])
mc_loss = tf.keras.callbacks.ModelCheckpoint('best_loss_model.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
model.fit(X_train1, y_train, batch_size=32, epochs=100, validation_data=(X_val1, y_val), callbacks=[mc_loss])

Epoch 1/100
534/534 [==============================] - ETA: 0s - loss: 167158.5469 - accuracy: 0.9227 - Recall: 0.3620 - Precision: 0.3012 - AUC: 0.4993
Epoch 1: val_loss improved from inf to 504.11212, saving model to best_loss_model.h5
534/534 [==============================] - 130s 239ms/step - loss: 167158.5469 - accuracy: 0.9227 - Recall: 0.3620 - Precision: 0.3012 - AUC: 0.4993 - val_loss: 504.1121 - val_accuracy: 0.9430 - val_Recall: 0.3594 - val_Precision: 0.4428 - val_AUC: 0.4856
Epoch 2/100
534/534 [==============================] - ETA: 0s - loss: 69.8041 - accuracy: 0.9461 - Recall: 0.3829 - Precision: 0.4812 - AUC: 0.5066
Epoch 2: val_loss improved from 504.11212 to 0.14536, saving model to best_loss_model.h5
534/534 [==============================] - 134s 252ms/step - loss: 69.8041 - accuracy: 0.9461 - Recall: 0.3829 - Precision: 0.4812 - AUC: 0.5066 - val_loss: 0.1454 - val_accuracy: 0.9598 - val_Recall: 0.3312 - val_Precision: 0.7632 - val_AUC: 0.5284
Epoch 3/100
534/53

KeyboardInterrupt: 